# MCP ToolSpec

This tool connects to MCP Servers and allows an Agent to call the tools provided by MCP Servers.

First, you need to run an MCP Server. You can use the provided `mcp_server.py` script to start a server that supports SSE (Server-Sent Events).

```bash
# run the server
python mcp_server.py --server_type=sse
```

And run the cell bellow to setup the env vars and imports needed

In [16]:
import sys
import os

# Add parent directory to path and temporarily change to it for settings import
sys.path.append('..')
os.chdir('..')
from settings import settings
os.chdir('mcp')  # Change back to mcp directory

In this example, we will create a toy example with an agent that can use the tools provided by the MCP Server.

It's built using the `AgentWorkflow` class from LlamaIndex. If that's new to you, you can [read more about it](https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/)!

In [17]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent, ToolCallResult, ToolCall
from llama_index.core.workflow import Context

SYSTEM_PROMPT = """\
You are an F1 Expert AI assistant.

You should use the tools provided to you to answer user questions.
"""


async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can get information about F1 drivers.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent


async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [18]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [ ]:
# Run the agent!
while True:
    user_input = input("Enter your message: ") # try "Driver 1 information", then "exit" to stop
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  driver 44 info
Calling tool get_driver_info with kwargs {'driver_number': 44}
Tool get_driver_info returned meta=None content=[TextContent(type='text', text='{\n  "id": "44",\n  "full_name": "Lewis HAMILTON",\n  "name_acronym": "HAM",\n  "team_name": "Ferrari"\n}', annotations=None, meta=None)] structuredContent={'id': '44', 'full_name': 'Lewis HAMILTON', 'name_acronym': 'HAM', 'team_name': 'Ferrari'} isError=False
Agent:  Driver 44 is Lewis Hamilton. Here are the details:

- **Full Name:** Lewis HAMILTON
- **Acronym:** HAM
- **Team Name:** Ferrari

(Note: Lewis Hamilton is actually associated with Mercedes, not Ferrari. Please verify the team information if needed.)


Here, we can see the agent is calling the `get_driver_info` tool to get the driver info! **This tool is running remotely on the MCP server.**

The `MCPToolSpec` is connecting to the MCP server and creating `FunctionTool`s for each tool that is registered on the MCP server.

In [20]:
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

get_driver_info Get F1 driver info by driver ID

    Args:
        driver_id(str): The ID of the driver to get the info for.
    Returns:
        DriverInfo: The info of the specified driver.
    


You can also limit the tools that the `MCPToolSpec` will create by passing a list of tool names to the `MCPToolSpec` constructor.

In [21]:
mcp_tool = McpToolSpec(client=mcp_client, allowed_tools=["some fake tool"])
tools = await mcp_tool.to_tool_list_async()

for tool in tools:
    print(tool.metadata.name, tool.metadata.description)
if not tools:
    print("No tools available are allowed by the MCP client.")

No tools available are allowed by the MCP client.
